In [3]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# ============================================================
# CONFIGURATION
# ============================================================
MODEL_PATH = "../../models/FC211042_Heshani/best_model.keras"      # Path to your trained model
IMG_SIZE = 160                       # Must match your training size
CLASSES = ["angry", "happy", "sad", "surprise", "neutral"]  # Based on your project

# ============================================================
# LOAD MODEL
# ============================================================
print("Loading model...")
model = tf.keras.models.load_model(MODEL_PATH)
print("Model loaded successfully!")

# ============================================================
# LOAD FACE DETECTOR
# ============================================================
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)
if face_cascade.empty():
    raise IOError("Haarcascade file not found!")

# ============================================================
# OPEN WEBCAM
# ============================================================
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot access webcam")

print("Real-time emotion detection started — press 'q' to quit")

# ============================================================
# MAIN LOOP
# ============================================================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale for detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Draw bounding box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Extract face ROI
        face_img = frame[y:y + h, x:x + w]
        face_resized = cv2.resize(face_img, (IMG_SIZE, IMG_SIZE))
        face_array = np.expand_dims(face_resized / 255.0, axis=0)

        # Predict emotion
        preds = model.predict(face_array, verbose=0)[0]
        label = CLASSES[np.argmax(preds)]
        confidence = np.max(preds) * 100

        # Display label above the box
        text = f"{label} ({confidence:.1f}%)"
        cv2.putText(
            frame, text, (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2
        )

    # Show the frame
    cv2.imshow("Real-Time Emotion Detection", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ============================================================
# CLEANUP
# ============================================================
cap.release()
cv2.destroyAllWindows()
print("Session ended.")


Loading model...


c:\Users\heshm\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 192 variables whereas the saved optimizer has 196 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model loaded successfully!
Real-time emotion detection started — press 'q' to quit


KeyboardInterrupt: 